In [1]:
##########
# This script is used to separate out the dataset into different folders
##########

import torchvision.models as models

import os

import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import FashionMNIST
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
import pandas as pd
import torch.utils.data.dataset as dataset
import re
import matplotlib.patches as patches
import shutil

In [35]:
##### For all paths
csv_path = "./DL_info.csv"
Image_slices_dir = "/home/weilunhuang/course/DL/project/faster-rcnn-pytorch/data/JPEGImages"
Anno_slices_dir = "/home/weilunhuang/course/DL/project/faster-rcnn-pytorch/data/Annotations"
ImageSet_slices_dir = "/home/weilunhuang/course/DL/project/faster-rcnn-pytorch/data/ImageSets/Main"

root_dir="/home/weilunhuang/course/DL/project/Data"

#create dict for 8 classes
dest_dir={};
for i in range(1,9):
    sub_path="/class"+str(i);
    dest_dir[i]={};
    #3 folders for JPEG, Annotations, ImageSets 
    dest_dir[i]['jpeg']=root_dir+sub_path+'/JPEGImages';
    dest_dir[i]['annotation']=root_dir+sub_path+'/Annotations';
    dest_dir[i]['imagesets']=root_dir+sub_path+'/ImageSets/Main';
print(dest_dir)
    
        

{1: {'jpeg': '/home/weilunhuang/course/DL/project/Data/class1/JPEGImages', 'annotation': '/home/weilunhuang/course/DL/project/Data/class1/Annotations', 'imagesets': '/home/weilunhuang/course/DL/project/Data/class1/ImageSets/Main'}, 2: {'jpeg': '/home/weilunhuang/course/DL/project/Data/class2/JPEGImages', 'annotation': '/home/weilunhuang/course/DL/project/Data/class2/Annotations', 'imagesets': '/home/weilunhuang/course/DL/project/Data/class2/ImageSets/Main'}, 3: {'jpeg': '/home/weilunhuang/course/DL/project/Data/class3/JPEGImages', 'annotation': '/home/weilunhuang/course/DL/project/Data/class3/Annotations', 'imagesets': '/home/weilunhuang/course/DL/project/Data/class3/ImageSets/Main'}, 4: {'jpeg': '/home/weilunhuang/course/DL/project/Data/class4/JPEGImages', 'annotation': '/home/weilunhuang/course/DL/project/Data/class4/Annotations', 'imagesets': '/home/weilunhuang/course/DL/project/Data/class4/ImageSets/Main'}, 5: {'jpeg': '/home/weilunhuang/course/DL/project/Data/class5/JPEGImages', '

In [36]:
#### Optional, remove it later, put the same into dataset class
df = pd.read_csv(csv_path)          # The DL_info.csv file path



test_df = df[df['Train_Val_Test']==3];
print("total len of test dataset")
print(len(test_df.index))


total_len=0;
df_dic={};

for i in range(1,9):
    df_dic[i]=test_df[df['Coarse_lesion_type']==i];
    print(len(df_dic[i].index));
    total_len+=len(df_dic[i].index);
print(total_len==len(test_df.index));


total len of test dataset
4927
108
1173
864
700
1100
234
339
409
True


/home/weilunhuang/miniconda2/envs/dl/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  from ipykernel import kernelapp as app


In [41]:
file_name = os.path.join(Anno_slices_dir, df_dic[1].iloc[0]['File_name'])
file_name,_=os.path.splitext(file_name)
file_name=file_name+'.xml'
print(file_name)
# shutil.copy(file_name, dest_dir[1]['jpeg'])

/home/weilunhuang/course/DL/project/faster-rcnn-pytorch/data/Annotations/000193_01_02_469/.xml


In [45]:
#separate images and annotations
for i in range(1,9):
    for j in range(len(df_dic[i].index)):
        if j>0:
            break;
        #jpeg
        file_name = os.path.join(Image_slices_dir, df_dic[i].iloc[j]['File_name'])        
        print (file_name)
        shutil.copy(file_name, dest_dir[i]['jpeg'])
        
        #xml
        file_name = os.path.join(Anno_slices_dir, df_dic[i].iloc[j]['File_name'])
        file_name,_=os.path.splitext(file_name)
        file_name=file_name+'.xml'
        print(file_name);
        shutil.copy(file_name, dest_dir[i]['annotation'])



/home/weilunhuang/course/DL/project/faster-rcnn-pytorch/data/JPEGImages/000193_01_02_469.png
/home/weilunhuang/course/DL/project/faster-rcnn-pytorch/data/Annotations/000193_01_02_469.xml
/home/weilunhuang/course/DL/project/faster-rcnn-pytorch/data/JPEGImages/000039_01_01_067.png
/home/weilunhuang/course/DL/project/faster-rcnn-pytorch/data/Annotations/000039_01_01_067.xml
/home/weilunhuang/course/DL/project/faster-rcnn-pytorch/data/JPEGImages/000001_01_01_109.png
/home/weilunhuang/course/DL/project/faster-rcnn-pytorch/data/Annotations/000001_01_01_109.xml
/home/weilunhuang/course/DL/project/faster-rcnn-pytorch/data/JPEGImages/000016_01_01_008.png
/home/weilunhuang/course/DL/project/faster-rcnn-pytorch/data/Annotations/000016_01_01_008.xml
/home/weilunhuang/course/DL/project/faster-rcnn-pytorch/data/JPEGImages/000030_01_01_223.png
/home/weilunhuang/course/DL/project/faster-rcnn-pytorch/data/Annotations/000030_01_01_223.xml
/home/weilunhuang/course/DL/project/faster-rcnn-pytorch/data/JPEG

In [49]:
#imagesets, for test dataset
for i in range(1,9):
    if i>1:
        break;
    path=dest_dir[i]['imagesets']+'/test.txt'
    file = open(path,"w") ;
    for j in range(len(df_dic[i].index)):
        idx=df_dic[i].iloc[j]['File_name'];
        idx,_=os.path.splitext(idx)
        file.write(idx+'\n');
    file.close();


In [48]:
file_name,_=os.path.splitext(df_dic[1].iloc[0]['File_name'])
print(file_name)

000193_01_02_469
